In [2]:
import torch
PT_FILE_PATH = '/home/wjh/DC/data/VOC2007/VOC2007.pt'
data = torch.load(PT_FILE_PATH)

images_train = data['images_train'] # 假设是 (N, C, H, W) 格式的 Tensor
labels_train = data['labels_train'] # 假设是 (N, NUM_CLASSES) 格式的 Multi-hot Tensor (float)

print(labels_train.shape)

print(labels_train[:10])

torch.Size([5011, 20])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
     

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# --- 配置参数 ---
# 数据集文件路径 (请根据您的实际路径修改)
DATA_FILES = {
    'train': '/home/wjh/NCFM/dataset/reuters/training_data.csv',
    'test': '/home/wjh/NCFM/dataset/reuters/test_data.csv',
    'validation': '/home/wjh/NCFM/dataset/reuters/val_data.csv'
}
datasets = load_dataset("csv", data_files=DATA_FILES)
print(datasets)

Generating train split: 6769 examples [00:00, 54546.73 examples/s]
Generating test split: 3019 examples [00:00, 69371.59 examples/s]
Generating validation split: 1000 examples [00:00, 50686.45 examples/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 6769
    })
    test: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 3019
    })
    validation: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 1000
    })
})


In [4]:
print(datasets['train'][0]['labels'])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
